# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
base_df = pd.read_csv('../WeatherPy/city_weather_data.csv',index_col=0)
base_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,New Norfolk,75,AU,1584487459,49,-42.78,147.06,71.01,10.29
1,Atuona,1,PF,1584487323,73,-9.80,-139.03,82.94,10.04
2,Brownwood,90,US,1584487460,64,31.71,-98.99,75.20,9.17
3,Butaritari,0,KI,1584487049,75,3.07,172.79,83.35,12.68
4,Upata,90,VE,1584487460,66,8.01,-62.40,77.54,6.64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
base_df = base_df.dropna()
locations = base_df[["Lat", "Lng"]].astype(float)
humidity = base_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = base_df.loc[base_df['Cloudiness'] < 10]
ideal_df = ideal_df.loc[ideal_df['Humidity'] < 40]
ideal_df = ideal_df.loc[ideal_df['Max Temp'] < 85]
ideal_df = ideal_df.loc[ideal_df['Max Temp'] > 50]
ideal_df = ideal_df.loc[ideal_df['Wind Speed'] < 10]

In [7]:
ideal_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
142,Portland,1,US,1584487197,29,45.52,-122.68,62.01,9.17
220,Malakal,8,SS,1584487512,14,9.53,31.66,81.21,9.31
457,Merta,8,IN,1584487567,36,26.65,74.03,66.61,6.96
491,Coihueco,0,CL,1584487574,33,-36.62,-71.83,75.20,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(inplace = True, drop = True)
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Portland,1,US,1584487197,29,45.52,-122.68,62.01,9.17,
1,Malakal,8,SS,1584487512,14,9.53,31.66,81.21,9.31,
2,Merta,8,IN,1584487567,36,26.65,74.03,66.61,6.96,
3,Coihueco,0,CL,1584487574,33,-36.62,-71.83,75.20,8.05,


In [9]:
target_coordinates = ","

params = {"location": target_coordinates,
    "radius": 5000,
    "type": 'lodging',
    "key": g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [10]:
params['location']= f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
response = requests.get(base_url,params=params).json()
pprint(response['results'][0]['name'])

'Portland Marriott City Center'


In [11]:
hotels = []
for x in range(len(hotel_df["City"])):
    params['location']= f"{hotel_df['Lat'][x]},{hotel_df['Lng'][x]}"
    response = requests.get(base_url,params=params).json()
    hotels.append(response['results'][0]['name'])
hotel_df["Hotel Name"] = hotels

In [12]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Portland,1,US,1584487197,29,45.52,-122.68,62.01,9.17,Portland Marriott City Center
1,Malakal,8,SS,1584487512,14,9.53,31.66,81.21,9.31,Nile Palace Hotel
2,Merta,8,IN,1584487567,36,26.65,74.03,66.61,6.96,V Resorts Ganthiya
3,Coihueco,0,CL,1584487574,33,-36.62,-71.83,75.20,8.05,Frutillares Coihueco


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))